In [1]:
!python -V


Python 3.6.8


In [35]:
import spacy
from bs4 import BeautifulSoup
import json
import re
from nltk.tokenize import wordpunct_tokenize, word_tokenize

In [3]:
with open('datasets/foodbase/FoodBase_curated.xml', 'r') as f:
    data = f.read()

Bs_data = BeautifulSoup(data, "xml")

b_doc = Bs_data.find_all('document')


In [4]:
print(type(b_doc[0]))

<class 'bs4.element.Tag'>


In [5]:
b_annotations = b_doc[0].find_all("annotation")
print(b_annotations)

[<annotation id="1">
<location length="12" offset="3"/>
<text>cream cheese</text>
<infon key="semantic_tags"> AG.01.e [Dairy produce];AG.01.e.02 [Cheese];AG.01.n [Dishes and prepared food];AG.01.n.18 [Preserve];</infon>
</annotation>, <annotation id="2">
<location length="4" offset="6"/>
<text>beef</text>
<infon key="semantic_tags"> AG.01.d.03 [Beef];</infon>
</annotation>, <annotation id="3">
<location length="6" offset="8"/>
<text>olives</text>
<infon key="semantic_tags"> AG.01.h.01.e [Fruit containing stone];</infon>
</annotation>, <annotation id="4">
<location length="5" offset="10"/>
<text>onion</text>
<infon key="semantic_tags"> AG.01.h.02.e [Onion/leek/garlic];</infon>
</annotation>, <annotation id="5">
<location length="20" offset="13"/>
<text>Worcestershire sauce</text>
<infon key="semantic_tags"> AG.01.h [Fruit and vegetables];AG.01.l.04 [Sauce/dressing];</infon>
</annotation>, <annotation id="6">
<location length="7" offset="63"/>
<text>walnuts</text>
<infon key="semantic_ta

In [51]:
def get_all_occurences(text, doc):
    return [m.start() for m in re.finditer(text, doc)]

# def get_tokens(sent):
#     tokens = word_tokenize(sent)
#     token_index = []
#     cur_indx = 0
#     for i in range(0, len(tokens)-1):
#         token = tokens[i]
#         token_index.append(cur_indx)
#         cur_indx += len(token) + (0 if tokens[i+1] in [',', '.'] else 1)
    
#     return tokens, token_index

def get_tokens(txt):
    tokens = word_tokenize(txt)
    offset = 0
    token_indx_tuples = []
    for token in tokens:
        offset = txt.find(token, offset)
        token_indx_tuples.append((token, offset))
        offset += len(token)
    
    return token_indx_tuples


In [61]:
data = []
i = 0
for docs in b_doc:
    infon = docs.find("infon", {"key": "full_text"})
    document = infon.get_text().strip()
    data.append([])
    data[i].append(document)
    data[i].append({'entities': []})
    current_end = 0
    token_indx_tuples = get_tokens(document)
    for anots in docs.find_all("annotation"):
        word = anots.find("text").get_text()
        place = anots.find("location")
        offset = place["offset"] # find the first occurence of this
        token, starting_index = token_indx_tuples[int(offset) - 1]
        length = place["length"]
        end = int(starting_index) + int(length)
        # print(document[starting_index:end])
        data[i][1]["entities"].append([starting_index, end, "INGREDIENT"])
        current_end = end
    i+=1
    # break
    

In [62]:
with open("FoodBase_curated_fixed.json", 'w') as outfile:
    json.dump(data, outfile)

In [63]:
with open("FoodBase_curated_fixed.json") as json_file:
    data = json.load(json_file)
print(data[0])

['Mix the cream cheese, beef, olives, onion, and Worcestershire sauce together in a bowl until evenly blended. Keeping the mixture in the bowl, scrape it into a semi-ball shape. Cover, and refrigerate until firm, at least 2 hours. Place a large sheet of waxed paper on a flat surface. Sprinkle with walnuts. Roll the cheese ball in the walnuts until completely covered. Transfer the cheese ball to a serving plate, or rewrap with waxed paper and refrigerate until needed.', {'entities': [[8, 20, 'INGREDIENT'], [22, 26, 'INGREDIENT'], [28, 34, 'INGREDIENT'], [36, 41, 'INGREDIENT'], [47, 67, 'INGREDIENT'], [297, 304, 'INGREDIENT'], [315, 326, 'INGREDIENT'], [334, 341, 'INGREDIENT'], [381, 392, 'INGREDIENT']]}]


In [69]:
doc, entites = data[2]
for left, right, label in entites['entities']:
    print(doc[left:right])

water
sun-dried tomatoes
water
cream cheese
garlic
provolone cheese
pesto
cream cheese
garlic mixture
sun-dried tomatoes
provolone
pesto
cream cheese
garlic mixture
sun-dried tomatoes
provolone cheese
excess oil
